In [9]:
from chain_builder import get_sentence
import cPickle as pickle
import tweepy
from datetime import date, timedelta
from keys import keys

In [10]:
with open('Pickles/forward_dict.pkl') as f:
    f_dict = pickle.load(f)

with open('Pickles/reversed_dict.pkl') as f:
    r_dict = pickle.load(f)

In [7]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweet_replies = []

today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)

# ******************************************************
# Queries Popular Tweets directed at Candidates or using
# well-known Hashtags and responds to all queried tweets.
# ******************************************************
hashtags = ["#FeelTheBern OR #ImWithHer OR #MakeAmericaGreatAgain OR #Trump2016 \
OR @BernieSanders OR @HillaryClinton OR @realDonaldTrump"]

top_tweets = tweepy.Cursor(api.search, q=hashtags, count=100, result_type='popular', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in top_tweets:
    if status.favorite_count > 1000:
        tweet_replies.append(status)

# ***************************************************
# Queries Tweets directed at or in reply to
# FeelTheBern_Bot and responds to all queried tweets.
# ***************************************************
bernbot_query = "to:FeelTheBern_Bot OR @FeelTheBern_Bot"

bernbot_tweets = tweepy.Cursor(api.search, q=bernbot_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in bernbot_tweets:
    tweet_replies.append(status)

# ************************************************
# Queries Tweets from Presidential Candidates and
# responds to those with greater than 500 likes.
# ************************************************
candidate_query = "from:BernieSanders OR from:HillaryClinton OR from:realDonaldTrump"

candidate_tweets = tweepy.Cursor(api.search, q=candidate_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in candidate_tweets:
    if status.favorite_count > 500:
        tweet_replies.append(status)


# DROPS ANY DUPLICATE TWEETS SO IT DOES NOT RESPOND TWICE
tweet_replies = set(tweet_replies)

In [11]:
best_tweets = []
for i, tweet in enumerate(tweet_replies):
    resp_length = 141
    text = tweet.text
    sn = tweet.user.screen_name
    response = 'Vote trump 2016 Macy'
    n = 0
    while (resp_length > 140 or 'Vote trump 2016' in response or 'Macy' in response) and n < 20: 
        response = get_sentence(text, f_dict, r_dict, randomness = 1) 
        response = "@{0} {1}".format(sn, response)
        resp_length = len(response)
        n += 1
    if 'Vote trump 2016' in response or 'Macy' in response or resp_length > 140:
        pass
    
    else: 
        print i, '----', tweet.id
        print '@', sn, text
        print response
        print '*************'
    best_tweets.append([response, tweet.id])

0 ---- 726079789182808065
@ realDonaldTrump Thank you Indiana! #Trump2016 https://t.co/WAuI0nRNzX
@realDonaldTrump Thank God he's coming back.
*************
1 ---- 726146494542282752
@ HillaryClinton An entire deck of woman cards? Deal us in. https://t.co/W5AHQO70Br
@HillaryClinton The six largest financial institutions issue more than two-thirds of all credit Cards; half of all mortgages.
*************
2 ---- 725843561757859841
@ BernieSanders What this campaign is about is whether we can mobilize our people to take back our government from a handful of billionaires.
@BernieSanders The reason that our campaign is doing well is that we are talking to the American people SHaredSacrifice.
*************
3 ---- 726181047323389952
@ BernieSanders Since the passage of NAFTA in 1994, Indiana has lost 113,000 good paying manufacturing jobs.
https://t.co/RUf9Yy9409
@BernieSanders Our job is to substantially increase the number of union jobs in this country.
*************
4 ---- 7260450737758289

In [ ]:
# best_list = []
# for i, tweet in enumerate(best_tweets):
#     if i in best_list:
# #         print i 
#         print tweet[0], tweet[1]
# #         s = api.update_status(tweet[0], tweet[1])